In [1]:
import pandas as pd

In [2]:
path='unclean_log.log'

If one tries to load this log with the option converters={'status': int, 'size': int, 'req_time': int}, an exception is thrown :

In [3]:
try:
    pd.read_csv(path,
                 sep=r'\s(?=(?:[^"]*"[^"]*")*[^"]*$)(?![^\[]*\])', 
                 engine='python', na_values=['-'], header=None,
                 usecols=[0, 3, 4, 5, 6, 7, 8,10],
                 names=['ip', 'time', 'request', 'status', 'size', 'referer',
                 'user_agent','req_time'],converters={'status': int, 'size': int, 'req_time': int})
except Exception as parse_error:
        print(parse_error)

invalid literal for int() with base 10: '"GET /agent/10577/bdl HTTP/1.1"'


Some line appears to be shifted, we have a request in the status column. Let's get the index of the lines where something wrong happens.

First, we load the dataframe without recasting :

In [4]:
df=pd.read_csv(path,
                 sep=r'\s(?=(?:[^"]*"[^"]*")*[^"]*$)(?![^\[]*\])', 
                 engine='python', na_values=['-'], header=None,
                 usecols=[0, 3, 4, 5, 6, 7, 8,10],
                 names=['ip', 'time', 'request', 'status', 'size', 'referer',
                 'user_agent','req_time'])

Now, the plan is straightforward : we try to recast the elements of a column using .apply, and log the indexes of the lines where that couldn't be achieved. For the sake of the example, we use the dictionary __wrong_lines__ to log these. The keys will be indexes of wrong lines, the values correspond to what couldn't be recasted.

However, this is tricky since a row "forgets" its index. Hence, we temporarily add a column 'index' to the dataframe using the built in function __reset_index__ .

In [5]:
wrong_lines={}
def convert_int_feedback_index(row,col):
    #function to be used with .apply
    try:
        ans = int(row[col])
    except:
        wrong_lines[row['index']] = row[col]
        ans= pd.np.nan
    return ans

def convert_log_with_feedback(df,col):
    wrong_lines.clear()
    df_index=df.reset_index()
    df_index[col]=df_index.apply(convert_int_feedback_index,axis=1,col=col)
    if len(wrong_lines.keys()) != 0:
        print("Warning! Some lines couldn't be converted properly")
    df_index.pop('index')
    return df_index

In [6]:
df = convert_log_with_feedback(df,'status')

Warning! Some lines couldn't be converted properly


Yay! Let's have a look at our dictionary :

In [7]:
wrong_lines

{249242: '"GET /agent/10577/bdl HTTP/1.1"',
 249245: '"GET /agent/4231/bdl HTTP/1.1"',
 249246: '"GET /agent/11344/bdl HTTP/1.1"',
 249247: '"GET /agent/5997/bdl HTTP/1.1"',
 249248: '"GET /agent/6862/bdl HTTP/1.1"',
 249249: '"GET /bdl/1014982/updated_at?datetime=2018-09-19%2020:29:51 HTTP/1.1"',
 249251: '"GET /agent/6383/bdl HTTP/1.1"',
 249255: '"GET /agent/3933/bdl HTTP/1.1"',
 249256: '"GET /agent/5995/bdl HTTP/1.1"',
 249257: '"GET /agent/6988/bdl HTTP/1.1"',
 249260: '"GET /agent/11344/bdl HTTP/1.1"',
 249265: '"GET /agent/10520/bdl HTTP/1.1"',
 249266: '"GET /agent/5232/bdl HTTP/1.1"',
 249267: '"GET /agent/8019/bdl HTTP/1.1"',
 249268: '"GET /agent/7418/bdl HTTP/1.1"',
 249269: '"GET /agent/10660/bdl HTTP/1.1"',
 249275: '"GET /agent/4868/bdl HTTP/1.1"',
 249276: '"GET /agent/4528/bdl HTTP/1.1"',
 249278: '"GET /agent/6442/bdl HTTP/1.1"',
 249280: '"GET /agent/7505/bdl HTTP/1.1"',
 249282: '"GET /agent/7818/bdl HTTP/1.1"',
 249285: '"GET /agent/11344/bdl HTTP/1.1"',
 249286: 

Interesting, almost all lines between 249242 and 249451 are wrong. Let's have a look at two typycal wrong lines:

In [8]:
f=open(path)
lines=f.readlines()
print(lines[249242])
print(lines[249386])
f.close()

111.22.117.229, 111.22.117.229 - - [19/Sep/2018:22:17:40 +0200] "GET /agent/10577/bdl HTTP/1.1" 204 - "-" "okhttp/3.8.0" apibackend.site.fr 429282

111.22.116.184 "111.22.105.253, 111.22.105.253" - - [19/Sep/2018:22:20:22 +0200] "GET /agent/5983/bdl HTTP/1.1" 204 - "-" "okhttp/3.8.0" apibackend.site.fr 512517



There's a pattern here. Soon, we'll be able to load these unclean logs automatically!